# Scenario 2: A cross-functional team with one data scientist working on an ML model

MLflow setup:
- tracking server: yes, local server
- backend store: sqlite database
- artifacts store: local fliesystem

The experiments can be explored locally by accessing the local tracking server.

To run this example you need to launch the mlflow server locally by running the following command in your terminal:

```mlflow server --backend-store-uri sqlite:///backend.db```

In [6]:
import mlflow
mlflow.set_tracking_uri("http://127.0.0.1:5000")

In [7]:
print(f"tracking URI: '{mlflow.get_tracking_uri()}'")

tracking URI: 'http://127.0.0.1:5000'


In [8]:
mlflow.search_experiments()

[<Experiment: artifact_location='/Users/songlisakim/workspace/mlops-learning/mlops-zoomcamp/02-experiment-tracking/running-mlflow-by-scenarios/artifacts_local/2', creation_time=1748590925472, experiment_id='2', last_update_time=1748590925472, lifecycle_stage='active', name='random-forest-best-models', tags={}>,
 <Experiment: artifact_location='/Users/songlisakim/workspace/mlops-learning/mlops-zoomcamp/02-experiment-tracking/running-mlflow-by-scenarios/artifacts_local/1', creation_time=1748590193111, experiment_id='1', last_update_time=1748590193111, lifecycle_stage='active', name='random-forest-hyperopt', tags={}>,
 <Experiment: artifact_location='mlflow-artifacts:/0', creation_time=1748588377152, experiment_id='0', last_update_time=1748588377152, lifecycle_stage='active', name='Default', tags={}>]

In [9]:
from sklearn.linear_model import LogisticRegression
from sklearn.datasets import load_iris
from sklearn.metrics import accuracy_score

mlflow.set_experiment("my-experiment-1")

with mlflow.start_run():

    X, y = load_iris(return_X_y=True)

    params = {"C": 0.1, "random_state": 42}
    mlflow.log_params(params)

    lr = LogisticRegression(**params).fit(X, y)
    y_pred = lr.predict(X)
    mlflow.log_metric("accuracy", accuracy_score(y, y_pred))

    mlflow.sklearn.log_model(lr, artifact_path="models")
    print(f"default artifacts URI: '{mlflow.get_artifact_uri()}'")

2025/05/30 10:53:33 INFO mlflow.tracking.fluent: Experiment with name 'my-experiment-1' does not exist. Creating a new experiment.
2025/05/30 10:53:35 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


default artifacts URI: '/Users/songlisakim/workspace/mlops-learning/mlops-zoomcamp/02-experiment-tracking/running-mlflow-by-scenarios/artifacts_local/3/f21a765eda4a430a91577f2511ce42ef/artifacts'
🏃 View run merciful-donkey-961 at: http://127.0.0.1:5000/#/experiments/3/runs/f21a765eda4a430a91577f2511ce42ef
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/3


In [10]:
from mlflow.tracking import MlflowClient

client = MlflowClient("http://127.0.0.1:5000")

In [11]:
client.search_registered_models()

[<RegisteredModel: aliases={}, creation_timestamp=1748591306881, description='', last_updated_timestamp=1748591306898, latest_versions=[<ModelVersion: aliases=[], creation_timestamp=1748591306898, current_stage='None', description='', last_updated_timestamp=1748591306898, name='nyc-taxi-regressor', run_id='aa5f35e4e3c84ce8885b248803f277e6', run_link='', source='/Users/songlisakim/workspace/mlops-learning/mlops-zoomcamp/02-experiment-tracking/running-mlflow-by-scenarios/artifacts_local/2/aa5f35e4e3c84ce8885b248803f277e6/artifacts/model', status='READY', status_message=None, tags={}, user_id='', version='1'>], name='nyc-taxi-regressor', tags={}>]

In [12]:
run_id = client.search_runs(experiment_ids='1')[0].info.run_id
mlflow.register_model(
    model_uri=f"runs:/{run_id}/models",
    name='iris-classifier'
)

Successfully registered model 'iris-classifier'.
2025/05/30 10:57:01 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: iris-classifier, version 1
Created version '1' of model 'iris-classifier'.


<ModelVersion: aliases=[], creation_timestamp=1748591821623, current_stage='None', description='', last_updated_timestamp=1748591821623, name='iris-classifier', run_id='8693e7fddcdd4e06b8a2fbaa86cfc351', run_link='', source='/Users/songlisakim/workspace/mlops-learning/mlops-zoomcamp/02-experiment-tracking/running-mlflow-by-scenarios/artifacts_local/1/8693e7fddcdd4e06b8a2fbaa86cfc351/artifacts/models', status='READY', status_message=None, tags={}, user_id='', version='1'>